In [20]:
import pandas as pd
from plotly.offline import iplot
import plotly.graph_objs as go
import numpy as np
from sklearn import metrics
import math
import matplotlib.pyplot as plt
from collections import defaultdict
from functools import partial
from itertools import repeat
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve, auc
from keras.utils import to_categorical
from keras import models
from keras import layers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Dropout, Activation

In [21]:
workDir = "/Users/brandonmanley/Documents/nBody/"

#Import data
fname = workDir + "data/brutusSim/3body/brutus10_4_3.csv"
df = pd.read_csv(fname)
print(df.head())
with pd.option_context('mode.use_inf_as_null', True):
    df = df.dropna(axis=0)
dfShuffle = shuffle(df,random_state=42)
df.head()

   fileNum  eventID         m1         m2         m3        x1        x2  \
0      2.0  10001.0  97.246806  51.902602  34.969975  2.148653  5.191928   
1      2.0  10002.0  97.246806  51.902602  34.969975  2.148653  5.191928   
2      2.0  10003.0  97.246806  51.902602  34.969975  2.148653  5.191928   
3      2.0  10004.0  97.246806  51.902602  34.969975  2.148653  5.191928   
4      2.0  10005.0  97.246806  51.902602  34.969975  2.148653  5.191928   

         x3        y1        y2  ...    x3tEnd    y1tEnd    y2tEnd    y3tEnd  \
0  1.329325  7.995898 -2.326574  ...  1.327029  7.995693 -2.326470 -3.817666   
1  1.329325  7.995898 -2.326574  ...  1.324776  7.995478 -2.326364 -3.821388   
2  1.329325  7.995898 -2.326574  ...  1.322568  7.995252 -2.326258 -3.825084   
3  1.329325  7.995898 -2.326574  ...  1.320403  7.995016 -2.326151 -3.828751   
4  1.329325  7.995898 -2.326574  ...  1.318281  7.994770 -2.326042 -3.832392   

    dx1tEnd   dx2tEnd   dx3tEnd   dy1tEnd   dy2tEnd   dy3tEnd 

,fileNum,eventID,m1,m2,m3,x1,x2,x3,y1,y2,...,x3tEnd,y1tEnd,y2tEnd,y3tEnd,dx1tEnd,dx2tEnd,dx3tEnd,dy1tEnd,dy2tEnd,dy3tEnd
0,2.0,10001.0,97.246806,51.902602,34.969975,2.148653,5.191928,1.329325,7.995898,-2.326574,...,1.327029,7.995693,-2.326470,-3.817666,0.021263,-0.729756,-0.582272,-0.053816,0.026817,-0.956404
1,2.0,10002.0,97.246806,51.902602,34.969975,2.148653,5.191928,1.329325,7.995898,-2.326574,...,1.324776,7.995478,-2.326364,-3.821388,0.021690,-0.738115,-0.571053,-0.056467,0.027091,-0.949438
2,2.0,10003.0,97.246806,51.902602,34.969975,2.148653,5.191928,1.329325,7.995898,-2.326574,...,1.322568,7.995252,-2.326258,-3.825084,0.022117,-0.746468,-0.559841,-0.059118,0.027360,-0.942464
3,2.0,10004.0,97.246806,51.902602,34.969975,2.148653,5.191928,1.329325,7.995898,-2.326574,...,1.320403,7.995016,-2.326151,-3.828751,0.022543,-0.754815,-0.548637,-0.061769,0.027624,-0.935484
4,2.0,10005.0,97.246806,51.902602,34.969975,2.148653,5.191928,1.329325,7.995898,-2.326574,...,1.318281,7.994770,-2.326042,-3.832392,0.022968,-0.763157,-0.537440,-0.064420,0.027883,-0.928496


In [22]:
i_col = ["m1","m2","m3","x1", "x2", "x3", "y1", "y2", "y3",
		"dx1","dx2","dx3","dy1","dy2","dy3","tEnd"]
o_col = ["x1tEnd", "x2tEnd", "x3tEnd", "y1tEnd", "y2tEnd", "y3tEnd",
		"dx1tEnd", "dx2tEnd", "dx3tEnd", "dy1tEnd", "dy2tEnd", "dy3tEnd"]

X1 = df.as_matrix(columns=i_col)
y1 = df.as_matrix(columns=o_col)

X_train,X_test,y_train,y_test = train_test_split(X1,y1, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape,y_test.shape)

/home/llavezzo/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.

/home/llavezzo/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.



(742632, 16) (742632, 12)
(185658, 16) (185658, 12)


In [23]:
for i in y_train[0]:
    print(i)

4.119593923
6.427622086
-9.418013716
10.97865132
6.5292792010000005
-16.0771539
-0.20135184399999997
0.909095652
-1.554035885
-0.178778333
1.670345187
-2.092791147


In [26]:
network = models.Sequential()
network.add(layers.Dense(16,activation='relu',input_dim=16))
for i in range(10):
    network.add(layers.Dense(128,activation='relu'))
network.add(layers.Dense(12,activation='relu'))
network.compile(optimizer='adam',loss='mae',metrics=['accuracy'])
network.save_weights(workDir + '/weights/model_init.h5')

In [27]:
network.load_weights(workDir + '/weights/model_init.h5')
history = network.fit(X_train,y_train,
                      epochs=300,
                      batch_size=1000,
                      validation_data=(X_test,y_test), 
                      verbose = 1)

Train on 742632 samples, validate on 185658 samples
Epoch 1/300
742632/742632 [==============================] - 9s 13us/step - loss: 2.8727 - accuracy: 0.4671 - val_loss: 2.6528 - val_accuracy: 0.5618
Epoch 2/300
742632/742632 [==============================] - 9s 12us/step - loss: 2.5694 - accuracy: 0.5946 - val_loss: 2.4957 - val_accuracy: 0.6276
Epoch 3/300
742632/742632 [==============================] - 9s 13us/step - loss: 2.4926 - accuracy: 0.6294 - val_loss: 2.4738 - val_accuracy: 0.6415
Epoch 4/300
742632/742632 [==============================] - 9s 12us/step - loss: 2.4560 - accuracy: 0.6474 - val_loss: 2.4395 - val_accuracy: 0.6563
Epoch 5/300
742632/742632 [==============================] - 9s 13us/step - loss: 2.4280 - accuracy: 0.6608 - val_loss: 2.4132 - val_accuracy: 0.6685
Epoch 6/300
742632/742632 [==============================] - 10s 13us/step - loss: 2.4113 - accuracy: 0.6693 - val_loss: 2.3834 - val_accuracy: 0.6843
Epoch 7/300
742632/742632 [====================

KeyboardInterrupt: 